In [20]:
from torch.utils.data import DataLoader, SequentialSampler
from dataclass_paired_vanilla import PairedVanilla

# Test-Dataset erstellen (Pfad und Dictionaries müssen übereinstimmen)
precision = "allele"
embed_base_dir = f"../../data_10x/embeddings/paired/{precision}"


# Lade Dictionaries (falls benötigt)
traV_dict = {}  # Lade die Dictionaries entsprechend
traJ_dict = {}
trbV_dict = {}
trbJ_dict = {}
mhc_dict = {}

test_file_path = "../../data_10x/splitted_datasets/allele/paired/test.tsv"
test_dataset = PairedVanilla(test_file_path, embed_base_dir, traV_dict, traJ_dict, trbV_dict, trbJ_dict, mhc_dict)

# Test-Dataloader erstellen
SEQ_MAX_LENGTH = 200  # Passe dies an dein Training an
BATCH_SIZE = 1  # Evaluierung oft Batchgröße 1

def pad_collate(self, batch):
        epitope_embeddings, tra_cdr3_embeddings, trb_cdr3_embeddings = [], [], []
        v_alpha, j_alpha, v_beta, j_beta = [], [], [], []
        epitope_sequence, tra_cdr3_sequence, trb_cdr3_sequence = [], [], []
        mhc = []
        task = []
        labels = []

        for item in batch:
            epitope_embeddings.append(item["epitope_embedding"])
            epitope_sequence.append(item["epitope_sequence"])
            tra_cdr3_embeddings.append(item["tra_cdr3_embedding"])
            tra_cdr3_sequence.append(item["tra_cdr3_sequence"])
            trb_cdr3_embeddings.append(item["trb_cdr3_embedding"])
            trb_cdr3_sequence.append(item["trb_cdr3_sequence"])
            v_alpha.append(item["v_alpha"])
            j_alpha.append(item["j_alpha"])
            v_beta.append(item["v_beta"])
            j_beta.append(item["j_beta"])
            mhc.append(item["mhc"])
            task.append(item["task"])
            labels.append(item["label"])

        max_length = self.seq_max_length

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    sampler=SequentialSampler(test_dataset),
    num_workers=0,  # Keep as 0 for compatibility
    collate_fn=pad_collate
)

Unmapped TRA_CDR3 sequences: 0 []
Unmapped TRB_CDR3 sequences: 0 []
Unmapped Epitope sequences: 0 []


In [7]:
import os
print(os.path.abspath("./data_10x/splitted_datasets/allele/paired/test.tsv"))


/home/ubuntu/PA-Cancer-Immunotherapy-Transformer/BA_ZHAW/models/paired_vanilla/data_10x/splitted_datasets/allele/paired/test.tsv


In [25]:
import torch
from vanilla_model import VanillaModel  # Modellstruktur importieren

# Parameter wie beim Training definieren
EMBEDDING_SIZE = 1024
SEQ_MAX_LENGTH = 30  # Dies war die maximale Sequenzlänge im Training
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Eingebettete Dimensionen aus dem gespeicherten Modell
traV_embed_len = 118
traJ_embed_len = 113
trbV_embed_len = 132
trbJ_embed_len = 29
mhc_embed_len = 79

# Setze die Hyperparameter entsprechend dem Training
hyperparameters = {
    "optimizer": "sgd",
    "learning_rate": 5e-3,
    "weight_decay": 0.075,
    "dropout_attention": 0.3,
    "dropout_linear": 0.45,
}

# Modell erstellen
model = VanillaModel(
    EMBEDDING_SIZE, 
    SEQ_MAX_LENGTH, 
    DEVICE, 
    traV_embed_len, 
    traJ_embed_len, 
    trbV_embed_len, 
    trbJ_embed_len, 
    mhc_embed_len, 
    hyperparameters
)

# Modellgewichte laden
checkpoint = torch.load("VanillaModel.pth", map_location=DEVICE)
model.load_state_dict(checkpoint, strict=True)
model.eval()  # Setze das Modell in den Evaluierungsmodus

print("Modell erfolgreich geladen mit SEQ_MAX_LENGTH = 30.")


Modell erfolgreich geladen mit SEQ_MAX_LENGTH = 30.


/tmp/ipykernel_39679/2040780303.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("VanillaModel.pth", map_location=DEVICE)


In [ ]:
import torch
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from vanilla_model import VanillaModel  # Modellstruktur importieren

# Parameter wie beim Training definieren
EMBEDDING_SIZE = 1024
SEQ_MAX_LENGTH = 30
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Eingebettete Dimensionen
traV_embed_len = 118
traJ_embed_len = 113
trbV_embed_len = 132
trbJ_embed_len = 29
mhc_embed_len = 79

# Hyperparameter wie im Training
hyperparameters = {
    "optimizer": "sgd",
    "learning_rate": 5e-3,
    "weight_decay": 0.075,
    "dropout_attention": 0.3,
    "dropout_linear": 0.45,
}

# Modell erstellen
model = VanillaModel(
    EMBEDDING_SIZE, 
    SEQ_MAX_LENGTH, 
    DEVICE, 
    traV_embed_len, 
    traJ_embed_len, 
    trbV_embed_len, 
    trbJ_embed_len, 
    mhc_embed_len, 
    hyperparameters
)

# Modellgewichte laden
checkpoint = torch.load("VanillaModel.pth", map_location=DEVICE)
model.load_state_dict(checkpoint, strict=True)
model.eval()

# Test DataLoader
pad_collate_fn = PadCollate(SEQ_MAX_LENGTH).pad_collate
test_dataloader = DataLoader(
    test_dataset,
    batch_size=1,
    sampler=test_sampler,
    num_workers=NUM_WORKERS,
    collate_fn=pad_collate_fn
)

# Funktion zur Evaluierung
def evaluate_model_predictions(model, test_dataloader):
    model.eval()
    all_true_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in test_dataloader:
            inputs = {
                "epitope": batch["epitope_embedding"].to(DEVICE),
                "tra_cdr3": batch["tra_cdr3_embedding"].to(DEVICE),
                "trb_cdr3": batch["trb_cdr3_embedding"].to(DEVICE),
                "v_alpha": batch["v_alpha"].to(DEVICE),
                "j_alpha": batch["j_alpha"].to(DEVICE),
                "v_beta": batch["v_beta"].to(DEVICE),
                "j_beta": batch["j_beta"].to(DEVICE),
                "mhc": batch["mhc"].to(DEVICE),
            }
            true_labels = batch["label"].cpu().numpy()

            outputs = model(
                inputs["epitope"], 
                inputs["tra_cdr3"], 
                inputs["trb_cdr3"], 
                inputs["v_alpha"], 
                inputs["j_alpha"], 
                inputs["v_beta"], 
                inputs["j_beta"], 
                inputs["mhc"]
            ).squeeze(1)

            predictions = (torch.sigmoid(outputs) > 0.5).cpu().numpy()
            all_true_labels.extend(true_labels)
            all_predictions.extend(predictions)

    all_true_labels = np.array(all_true_labels)
    all_predictions = np.array(all_predictions)

    conf_matrix = confusion_matrix(all_true_labels, all_predictions)
    accuracy = accuracy_score(all_true_labels, all_predictions)
    precision = precision_score(all_true_labels, all_predictions, zero_division=0)
    recall = recall_score(all_true_labels, all_predictions, zero_division=0)

    print("Confusion Matrix:")
    print(conf_matrix)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

    print("\nBeispielhafte Vorhersagen (erste 20):")
    print("True Labels:     ", all_true_labels[:20])
    print("Predicted Labels:", all_predictions[:20])

# Evaluierung durchführen
evaluate_model_predictions(model, test_dataloader)


/tmp/ipykernel_39679/3319432659.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("VanillaModel.pth", map_location=DEVICE)


In [26]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

# Funktion zur Auswertung des Modells
def evaluate_predictions(model):
    # Extrahiere gespeicherte Vorhersagen und Labels
    test_predictions = torch.stack(model.test_predictions).squeeze(1).cpu().numpy()
    test_labels = torch.stack(model.test_labels).squeeze(1).cpu().numpy()
    
    # Berechnung der binären Vorhersagen (Schwellenwert: 0.5)
    binary_predictions = (test_predictions > 0.5).astype(int)
    
    # Metriken berechnen
    conf_matrix = confusion_matrix(test_labels, binary_predictions)
    accuracy = accuracy_score(test_labels, binary_predictions)
    precision = precision_score(test_labels, binary_predictions, zero_division=0)
    recall = recall_score(test_labels, binary_predictions, zero_division=0)

    # Ergebnisse anzeigen
    print("Confusion Matrix:")
    print(conf_matrix)
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    
    # Vorhersagen und Labels anzeigen (erste 20)
    print("\nBeispielhafte Vorhersagen (erste 20):")
    print("True Labels:     ", test_labels[:20])
    print("Predicted Labels:", binary_predictions[:20])

# Beispielaufruf
evaluate_predictions(model)


RuntimeError: stack expects a non-empty TensorList

In [1]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

def evaluate_loaded_model(model, dataloader):
    true_labels = []
    predicted_labels = []

    with torch.no_grad():  # Keine Gradientenberechnung erforderlich
        for data in dataloader:
            inputs, labels = data  # Passe dies ggf. an deine Datenstruktur an
            outputs = model(inputs.to(DEVICE))
            _, predicted = torch.max(outputs, 1)

            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    # Berechne Metriken
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, zero_division=0)
    recall = recall_score(true_labels, predicted_labels, zero_division=0)

    print("Confusion Matrix:")
    print(conf_matrix)
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")

# Beispiel für den Aufruf der Evaluierungsfunktion
# Annahme: test_dataloader wurde bereits erstellt, wie in deinem Trainingsskript
evaluate_loaded_model(model, test_dataloader)


NameError: name 'model' is not defined

In [ ]:
# Importieren der notwendigen Bibliotheken
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, SequentialSampler
from vanilla_model import VanillaModel
from dataclass_paired_vanilla import PairedVanilla
import os

In [19]:
# Parameter setzen
EMBEDDING_SIZE = 1024
SEQ_MAX_LENGTH = 30  # Maximal erlaubte Sequenzlänge
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Setzen Sie den Pfad zu den Testdaten und Embeddings
precision = "allele"
data_dir = "../../data_10x/splitted_datasets/allele/paired/"
embed_base_dir = "../../data_10x/embeddings/paired/allele"

test_file_path = f"{data_dir}/test.tsv"

# Hilfsfunktion: Spalte in Dictionary umwandeln
def column_to_dictionary(df, column_name):
    list_of_column = df[column_name].unique()
    return {item: index for index, item in enumerate(list_of_column)}

# Hilfsfunktion: Länge der Einbettungen berechnen
def get_embed_len(df, column_name):
    return len(df[column_name].unique())

# Daten laden und Dictionaries erstellen
df_test = pd.read_csv(test_file_path, sep="\t")
traV_dict = column_to_dictionary(df_test, "TRAV")
traJ_dict = column_to_dictionary(df_test, "TRAJ")
trbV_dict = column_to_dictionary(df_test, "TRBV")
trbJ_dict = column_to_dictionary(df_test, "TRBJ")
mhc_dict = column_to_dictionary(df_test, "MHC")

traV_embed_len = get_embed_len(df_test, "TRAV")
traJ_embed_len = get_embed_len(df_test, "TRAJ")
trbV_embed_len = get_embed_len(df_test, "TRBV")
trbJ_embed_len = get_embed_len(df_test, "TRBJ")
mhc_embed_len = get_embed_len(df_test, "MHC")

# Test-Dataset erstellen
test_dataset = PairedVanilla(test_file_path, embed_base_dir, traV_dict, traJ_dict, trbV_dict, trbJ_dict, mhc_dict)
print("Eintrag im Dataset:", test_dataset[0])

# Test-Dataloader erstellen
class PadCollate:
    def __init__(self, seq_max_length):
        self.seq_max_length = seq_max_length

    def pad_collate(self, batch):
        epitope_embeddings, tra_cdr3_embeddings, trb_cdr3_embeddings = [], [], []
        v_alpha, j_alpha, v_beta, j_beta = [], [], [], []
        epitope_sequence, tra_cdr3_sequence, trb_cdr3_sequence = [], [], []
        mhc, task, labels = [], [], []

        for item in batch:
            epitope_embeddings.append(item["epitope_embedding"])
            epitope_sequence.append(item["epitope_sequence"])
            tra_cdr3_embeddings.append(item["tra_cdr3_embedding"])
            tra_cdr3_sequence.append(item["tra_cdr3_sequence"])
            trb_cdr3_embeddings.append(item["trb_cdr3_embedding"])
            trb_cdr3_sequence.append(item["trb_cdr3_sequence"])
            v_alpha.append(item["v_alpha"])
            j_alpha.append(item["j_alpha"])
            v_beta.append(item["v_beta"])
            j_beta.append(item["j_beta"])
            mhc.append(item["mhc"])
            task.append(item["task"])

            # Labels sammeln
            if "Binding" in item:
                labels.append(int(item["Binding"]))  # Binding-Spalte verwenden
            else:
                print("Warnung: 'Binding' fehlt im Batch-Item:", item)

        max_length = self.seq_max_length

        # Debugging für Labels
        print("Labels Debugging:")
        print(labels)  # Zeigt die gesammelten Labels
        print("Label-Typ:", type(labels))  # Überprüfen Sie den Typ von `labels`

        # Konvertieren Sie `labels` zu einem Tensor
        try:
            labels = torch.tensor(labels, dtype=torch.int64)
        except Exception as e:
            print("Fehler bei der Konvertierung von Labels:", e)
            raise

        def pad_embeddings(embeddings):
            return torch.stack([
                torch.nn.functional.pad(embedding, (0, 0, 0, max_length - embedding.size(0)), "constant", 0)
                for embedding in embeddings
            ])

        epitope_embeddings = pad_embeddings(epitope_embeddings)
        tra_cdr3_embeddings = pad_embeddings(tra_cdr3_embeddings)
        trb_cdr3_embeddings = pad_embeddings(trb_cdr3_embeddings)

        v_alpha = torch.tensor(v_alpha, dtype=torch.int32)
        j_alpha = torch.tensor(j_alpha, dtype=torch.int32)
        v_beta = torch.tensor(v_beta, dtype=torch.int32)
        j_beta = torch.tensor(j_beta, dtype=torch.int32)
        mhc = torch.tensor(mhc, dtype=torch.int32)

        return {
            "epitope_embedding": epitope_embeddings,
            "tra_cdr3_embedding": tra_cdr3_embeddings,
            "trb_cdr3_embedding": trb_cdr3_embeddings,
            "v_alpha": v_alpha,
            "j_alpha": j_alpha,
            "v_beta": v_beta,
            "j_beta": j_beta,
            "mhc": mhc,
            "label": labels  # bleibt für Konsistenz in nachfolgendem Code
        }




pad_collate = PadCollate(SEQ_MAX_LENGTH).pad_collate

test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=1,
    sampler=test_sampler,
    collate_fn=pad_collate
)

hyperparameters = {}
hyperparameters["optimizer"] = "sgd"  # oder "adam", je nach Bedarf
hyperparameters["learning_rate"] = 5e-3
hyperparameters["weight_decay"] = 0.075
hyperparameters["dropout_attention"] = 0.3
hyperparameters["dropout_linear"] = 0.45

checkpoint_path = "VanillaModel.pth"
if not os.path.exists(checkpoint_path):
    raise FileNotFoundError(f"Checkpoint file not found: {checkpoint_path}")


# Modell erstellen und Gewichte laden
model = VanillaModel(EMBEDDING_SIZE, SEQ_MAX_LENGTH, DEVICE, traV_embed_len, traJ_embed_len, trbV_embed_len, trbJ_embed_len, mhc_embed_len, hyperparameters)
checkpoint = torch.load(checkpoint_path, map_location=DEVICE)
filtered_state_dict = {k: v for k, v in checkpoint.items() if k in model.state_dict() and model.state_dict()[k].size() == v.size()}
model.load_state_dict(filtered_state_dict, strict=False)
model.eval()
print("Model successfully loaded and set to evaluation mode.")

# Vorhersagen sammeln
predictions, labels = [], []
print("Labels before conversion:", labels)
print("Type of labels:", type(labels))

with torch.no_grad():
    for batch in test_dataloader:
        epitope_embedding = batch["epitope_embedding"].to(DEVICE)
        tra_cdr3_embedding = batch["tra_cdr3_embedding"].to(DEVICE)
        trb_cdr3_embedding = batch["trb_cdr3_embedding"].to(DEVICE)
        v_alpha = batch["v_alpha"].to(DEVICE)
        j_alpha = batch["j_alpha"].to(DEVICE)
        v_beta = batch["v_beta"].to(DEVICE)
        j_beta = batch["j_beta"].to(DEVICE)
        mhc = batch["mhc"].to(DEVICE)
        label = batch["label"].to(DEVICE)

        output = model(epitope_embedding, tra_cdr3_embedding, trb_cdr3_embedding, v_alpha, j_alpha, v_beta, j_beta, mhc)
        probs = torch.sigmoid(output)

        predictions.extend(probs.cpu().numpy())
        labels.extend(label.cpu().numpy())

# Visualisierung der Ergebnisse
plt.figure(figsize=(10, 6))
plt.scatter(range(len(predictions)), predictions, label="Predictions", alpha=0.6)
plt.scatter(range(len(labels)), labels, label="True Labels", alpha=0.6)
plt.xlabel("Samples")
plt.ylabel("Probability / Label")
plt.title("Predictions vs. True Labels")
plt.legend()
plt.show()

# Histogramm der Vorhersagen
plt.figure(figsize=(10, 6))
plt.hist([p for p, l in zip(predictions, labels) if l == 1], bins=20, alpha=0.5, label="Positive Class Predictions")
plt.hist([p for p, l in zip(predictions, labels) if l == 0], bins=20, alpha=0.5, label="Negative Class Predictions")
plt.xlabel("Predicted Probability")
plt.ylabel("Frequency")
plt.title("Prediction Distribution")
plt.legend()
plt.show()

# Ergebnisse speichern
results_df = pd.DataFrame({"Prediction": predictions, "Label": labels})
results_df.to_csv("prediction_results.csv", index=False)

from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score

roc_auc = roc_auc_score(labels, predictions)
ap_score = average_precision_score(labels, predictions)
accuracy = accuracy_score(labels, predicted_classes)

print(f"ROC-AUC: {roc_auc:.4f}, Average Precision: {ap_score:.4f}, Accuracy: {accuracy:.4f}")

# Speichern Sie die Metriken in einer Datei
metrics_df = pd.DataFrame({"Metric": ["ROC-AUC", "AP", "Accuracy"], "Value": [roc_auc, ap_score, accuracy]})
metrics_df.to_csv("evaluation_metrics.csv", index=False)


Unmapped TRA_CDR3 sequences: 0 []
Unmapped TRB_CDR3 sequences: 0 []
Unmapped Epitope sequences: 0 []
Eintrag im Dataset: {'epitope_embedding': tensor([[ 0.2024, -0.1784, -0.1524,  ..., -0.0598, -0.0730,  0.0883],
        [ 0.0946, -0.0478, -0.2511,  ...,  0.0762, -0.0088,  0.0973],
        [ 0.2836, -0.0068, -0.0652,  ..., -0.0279,  0.0056, -0.1149],
        ...,
        [ 0.0949,  0.0307,  0.0768,  ..., -0.0136, -0.0991, -0.0265],
        [ 0.1097, -0.0155, -0.0418,  ...,  0.2618, -0.0979,  0.0138],
        [ 0.0736, -0.0596,  0.1575,  ...,  0.1424, -0.1303, -0.1070]]), 'epitope_sequence': 'TTDPSFLGRY', 'tra_cdr3_embedding': tensor([[ 0.1556, -0.0179, -0.3081,  ...,  0.1541, -0.0970, -0.0399],
        [ 0.1623, -0.2130, -0.1466,  ..., -0.1051,  0.2095,  0.1247],
        [ 0.2351, -0.0936, -0.0034,  ...,  0.2117,  0.1583,  0.1143],
        ...,
        [ 0.0273,  0.2817, -0.1794,  ...,  0.0676, -0.1381, -0.0646],
        [-0.0170,  0.1087, -0.0216,  ...,  0.2675, -0.1181, -0.2048],
   

/tmp/ipykernel_39679/352622923.py:140: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=DEVICE)


Model successfully loaded and set to evaluation mode.
Labels before conversion: []
Type of labels: <class 'list'>
Warnung: 'Binding' fehlt im Batch-Item: {'epitope_embedding': tensor([[ 0.2024, -0.1784, -0.1524,  ..., -0.0598, -0.0730,  0.0883],
        [ 0.0946, -0.0478, -0.2511,  ...,  0.0762, -0.0088,  0.0973],
        [ 0.2836, -0.0068, -0.0652,  ..., -0.0279,  0.0056, -0.1149],
        ...,
        [ 0.0949,  0.0307,  0.0768,  ..., -0.0136, -0.0991, -0.0265],
        [ 0.1097, -0.0155, -0.0418,  ...,  0.2618, -0.0979,  0.0138],
        [ 0.0736, -0.0596,  0.1575,  ...,  0.1424, -0.1303, -0.1070]]), 'epitope_sequence': 'TTDPSFLGRY', 'tra_cdr3_embedding': tensor([[ 0.1556, -0.0179, -0.3081,  ...,  0.1541, -0.0970, -0.0399],
        [ 0.1623, -0.2130, -0.1466,  ..., -0.1051,  0.2095,  0.1247],
        [ 0.2351, -0.0936, -0.0034,  ...,  0.2117,  0.1583,  0.1143],
        ...,
        [ 0.0273,  0.2817, -0.1794,  ...,  0.0676, -0.1381, -0.0646],
        [-0.0170,  0.1087, -0.0216,  ...

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)